In [6]:
# For adding  airports information in airports table in sql database

import pandas as pd
import requests
from datetime import datetime, timedelta
from pytz import timezone
from keys import sql_password, RAPIAPI_KEY
from sqlalchemy import create_engine, text


#function to create connection string to make connection with Database
def create_connection_string():
  schema = "gans"
  host = "35.241.135.102"
  user = "root"
  password = sql_password
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# To read cities info from cities table
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# to get airports for the cities

    
def icao_airport_codes(cities_df):

  #assert len(latitudes) == len(longitudes)
  latitudes = cities_df['latitude']
  longitudes = cities_df['longitude']
  cities_id = cities_df['city_id']
  list_for_df = []
    
  for index, value in enumerate(latitudes):

    url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{value}/{longitudes[index]}/km/50/16"

    querystring = {"withFlightInfoOnly":"true"}

    headers = {
      "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
      "X-RapidAPI-Key": RAPIAPI_KEY
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    if response.status_code ==200:
        response_df = pd.json_normalize(response.json()['items'])
        response_df['city_id'] = cities_id[index]
        list_for_df.append(response_df)

  
  airports_df = (pd.concat(list_for_df, ignore_index=True))
  ret_airports_df = airports_df[['icao', 'name', 'timeZone', 'city_id']]
  ret_airports_df.rename(columns={"name": "airport_name"}, inplace=True)
  return ret_airports_df


def retrieve_and_save_airports():
    connection_string = create_connection_string()
    ret_cities_df = fetch_cities_data(connection_string)
    airports_df = icao_airport_codes(ret_cities_df)
    # airports_df.to_sql('airports',
    #               if_exists='append',
    #               con=connection_string,
    #               index=False)
    engine = create_engine(connection_string)
    for _, rows in airports_df.iterrows():
        icao = rows['icao']
        airport_name = rows['airport_name']
        timeZone = rows['timeZone']
        city_id = rows['city_id']
        with engine.connect() as connection:
            sql = text("""INSERT INTO airports (icao, airport_name, timeZone, city_id) 
            VALUES (:icao, :airport_name,:timeZone,:city_id) ON DUPLICATE KEY UPDATE 
            airport_name = :airport_name, timeZone = :timeZone, city_id = :city_id;""")
            
            connection.execute(sql,{"icao":icao, "airport_name": airport_name, "timeZone": timeZone, "city_id":city_id})
            
            connection.commit()
    return "Airports information retrived and saved in database"




In [10]:
retrieve_and_save_airports()

/var/folders/vs/m818tpl12dx9lgmg3w2kzqch0000gn/T/ipykernel_11294/3306542783.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ret_airports_df.rename(columns={"name": "airport_name"}, inplace=True)


'Airports information retrived and saved in database'